In [1]:
import json

def save_multiple_qa_to_jsonl(qa_list, filepath):
    """
    Save multiple question-answer pairs into a JSONL file at once.
    
    Args:
        qa_list (list of dicts): Each dict must have 'question' and 'answer' keys.
        filepath (str): Path to the .jsonl file.
    """
    with open(filepath, 'a', encoding='utf-8') as f:
        for qa in qa_list:
            json_line = json.dumps(qa, ensure_ascii=False)
            f.write(json_line + '\n')


In [8]:
import pandas as pd

def csv_to_jsonl(csv_path, jsonl_path):
    df = pd.read_csv(csv_path)
    qa_list = df.to_dict(orient='records')
    save_multiple_qa_to_jsonl(qa_list, jsonl_path)

# Example:
csv_to_jsonl("data.csv", "training_dataset.jsonl")


In [ ]:
import json

def save_qa_to_jsonl(question, answer, filepath):
    """
    Save a question-answer pair into a JSONL file, preserving formatting.
    
    Args:
        question (str): The natural language question or prompt.
        answer (str): The code snippet (answer).
        filepath (str): Path to the .jsonl file.
    """
    data = {
        "question": question,
        "answer": answer
    }
    
    with open(filepath, 'a', encoding='utf-8') as f:
        json_line = json.dumps(data, ensure_ascii=False)
        f.write(json_line + '\n')

# Example usage:
if __name__ == "__main__":
    q = """How do you allow any logged-in user to read global configuration in Rego?"""
    a = """team_role(subject, team_id) = role {
	subject_team := subject.teams[_]
	subject_team.id == team_id
	role := subject_team.role
}"""
    
    save_qa_to_jsonl(q, a, "training_dataset.jsonl")


In [ ]:
import requests
import json

ITM_url = "https://7708-35-184-143-188.ngrok-free.app/ask"
payload = {"question": "Servers are not allowed to expose the 'telnet' protocol"}
policyresponse = requests.post(ITM_url, json=payload)

opa_check_url = "http://fyp-opa-1:8181/v1/polices/" + str(random.randint(1000, 9999))
response = requests.put(url=opa_check_url, data=policyresponse, timeout=5)



{"question": "Servers are not allowed to expose the 'telnet' protocol"}


In [ ]:
import requests
import json
import random

ITM_url = "https://41d1-34-126-86-180.ngrok-free.app/ask"
opa_url_base = "http://localhost:8181/v1/polices/"
success_count = 0

# Read test inputs
with open("testing_data.txt", "r") as f:
    questions = [line.strip() for line in f if line.strip()]

policy_id = 1000

for question in questions:
    payload = {"question": question}

    try:
        # Get the generated policy
        policy_response = requests.post(ITM_url, json=payload)
        policy_response.raise_for_status()

        # Create a unique OPA policy name
        opa_check_url = opa_url_base + str(policy_id)

        # Submit to OPA
        response = requests.put(url=opa_check_url, data=policy_response, timeout=5)

        if response.status_code == 200:
            success_count += 1
        else:
            print(f"OPA error {response.status_code}: {response.text}")

    except Exception as e:
        print(f"Error with question '{question}': {e}")

    policy_id += 1

print(f"\n Total numbers of valid responses : {success_count}")